In [3]:
%pip install kagglehub
%pip install python-dotenv
%pip install -q -U google-generativeai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\luxma\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\luxma\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\luxma\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [35]:
import kagglehub
import pandas as pd

# Download latest version
path = kagglehub.dataset_download("jrobischon/wikipedia-movie-plots")

print("Path to dataset files:", path)

df = pd.read_csv(path+"\wiki_movie_plots_deduped.csv")
df.to_csv('movies.csv', index=False)

Path to dataset files: C:\Users\luxma\.cache\kagglehub\datasets\jrobischon\wikipedia-movie-plots\versions\1


In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load the dataset (assuming CSV format after kagglehub download)
df = pd.read_csv("movies.csv")

# Check for plot column and preprocess
df = df.drop(columns=["Director","Cast","Wiki Page"])
df = df[df["Origin/Ethnicity"]=="American"]
df = df.drop(columns=["Origin/Ethnicity"])
df = df.dropna(subset=["Plot"]).reset_index(drop=True)
df = df.sort_values(by='Release Year', ascending=False).head(500)

# Preview dataset
print(df.head())

# Create TF-IDF vectorizer
vectorizer = TfidfVectorizer(stop_words="english", max_df=0.8)
tfidf_matrix = vectorizer.fit_transform(df["Plot"])

# Recommender function
def recommend_movies(input_sentence, top_n=5):
    """
    Recommend top_n movies based on plot similarity to input_sentence.
    """
    input_vec = vectorizer.transform([input_sentence])
    # Use cosine similarity
    similarity_scores = cosine_similarity(input_vec, tfidf_matrix).flatten()
    
    # Get indices of top_n similar plots
    top_indices = similarity_scores.argsort()[-top_n:][::-1]
    
    recommendations = df.iloc[top_indices][["Title", "Release Year", "Plot"]]
    recommendations["Similarity Score"] = similarity_scores[top_indices]
    
    return recommendations

    

       Release Year                 Title                  Genre  \
17315          2017            Flatliners  horror, sci-fi, drama   
17346          2017                   LBJ       biography, drama   
17361          2017  Just Getting Started         action, comedy   
17376          2017        Phantom Thread                  drama   
17360          2017              I, Tonya       biography, drama   

                                                    Plot  
17315  A medical student, Courtney (Ellen Page), is o...  
17346  The story centers around the political upheava...  
17361  Defense lawyer Duke, who once defended the hea...  
17376  In 1954 London, renowned fashion designer Reyn...  
17360  In 1970s Portland, Oregon, three-year-old Tony...  


In [37]:
query = "I like action movies set in space"
top_recommendations = recommend_movies(query, top_n=5)

print("Top Movie Recommendations:\n")
print(top_recommendations)

Top Movie Recommendations:

                              Title  Release Year  \
17050                 Gods of Egypt          2016   
17100  Independence Day: Resurgence          2016   
17327                      Geostorm          2017   
17113     Ice Age: Collision Course          2016   
17251                     The Mummy          2017   

                                                    Plot  Similarity Score  
17050  In an alternate version of Egypt, the world is...          0.077022  
17100  Twenty years after the devastating alien invas...          0.062166  
17327  In 2019, following many catastrophic natural d...          0.054284  
17113  Scrat, trying to bury his acorn, accidentally ...          0.053552  
17251  In 1157 A.D., several English crusader knights...          0.048170  


In [26]:
import os
import google.generativeai as genai
# Used to securely store API key
api_key = os.getenv("api_key")
genai.configure(api_key=api_key)
model = genai.GenerativeModel('gemini-2.0-flash')
response = model.generate_content("Write a 5 sentence plot summary for a movie based on this request: "+query)
print(response.text)

recommend_movies(response.text, top_n=5)

A hardened space marshal, framed for a crime he didn't commit, escapes a maximum-security orbital prison. Hunted by relentless cyborg bounty hunters and the corrupt corporation that set him up, he must clear his name. He uncovers a conspiracy to weaponize a newly discovered alien artifact with devastating power.  Teaming up with a scrappy rogue pilot, he fights his way across the galaxy to reach the artifact first. In a final showdown, he confronts the corporation, preventing the weaponization and exposing their treachery to the universe. 



,Title,Release Year,Plot,Similarity Score
17364,Beyond Skyline,2017,"Mark Corley, a Los Angeles police detective, h...",0.083972
17100,Independence Day: Resurgence,2016,Twenty years after the devastating alien invas...,0.062281
17241,Alien: Covenant,2017,"In a prologue, business magnate Peter Weyland ...",0.038560
17327,Geostorm,2017,"In 2019, following many catastrophic natural d...",0.032801
17050,Gods of Egypt,2016,"In an alternate version of Egypt, the world is...",0.030075
